# DSC440 Programming Project
## Weihong Qi
### Oct.6 2022

## 1 Introduction

In this project, I develop and apply the Apriori and FP-Growth algorithms to the test data and evaluate the performance of both algorithms. For each algorithm, I firstly describe the algorithm in a natural language framework to show each step. Then I present the Python code to build and execute the main method of the algorithm. To test the performance of algorithms, I use the Use the UCI Adult Census Dataset (UCI Machine Learning Repository: Adult Data Set, n.d.) to test the methods and their running time. In the end, I compare the performance of the two algorithms.

Section 2.1 and 2.2 of this report presents the algorithm, code and performance evaluation of Apriori. In Section 2.3, I develop the Hash-based Apriori and compare the efficiency between the two Apriori algorithms. Section 3 presents the algorithm, code and performance evaluation of the FP-Growth algorithm. I comprare the efficiency between Apriori and FP-Growth in Section 4.

In [2]:
#import libraries
import pandas as pd
from collections import defaultdict
import time
from itertools import combinations
from numpy import int32

## 2 Apriori
The Apriori algorithm utilizes the Apriori property, which states as an itemset to be frequent, any of its non-empty subset must also be frquent. The algorithm involves a joint step and a prune step. Slightly differnt with Han, Kamber, and Pei (2012), in my algorithm, for each itemset, I prune the every subset first, and then check whetehr the itemset is frequent. Thus the algorithm to find a frequent  k-itemset is stated as follows:

- Prune step: For each k-itemset in the data, for each non-empty seubset, check if the subset is in $L_1$, $L_2$, ..., $L_{k-1}$. If the subset is frequent, check the next subset; if the subset is not frequent, the k-item set is not frequent, and move to check the next k-itemset.
- Scan all the tuples in the database and record the support counts of the k-item set. Check the whether the support count is greater than minimum support to determine whether this this itemset is frequent.

In the next subsection, I present the code to execute the above algorithm. I test the algorithm with the UCI Adult Census Dataset.

### 2.1 Main method

In [4]:
### Main method of Apriori ###

def apriori(data, min_sup):
    start_time = time.time()
    # read data
    with open(data) as df:
        lines = df.readlines()
    df.close()

    n_lines = sum(1 for line in open(data))  # generate the number of lines
    # print ("There are ", n_lines, "tuples in this data")
    # print (min_sup)

    # frequent 1-itemset L1
    # generate C1
    supp_counts1 = defaultdict(int)

    for line in lines:
        for item in line.split(', '):
            supp_counts1[item] += 1

    # generate L1
    frequent_item1 = set()
    for index in supp_counts1:
        if supp_counts1[index]/n_lines > min_sup:
            frequent_item1.add(index)
    print("Frequent 1-item set: \n", frequent_item1)

    # frequent 2-itemset L2
    supp_counts2 = defaultdict(int)

    # generate C2 based on L1
    for line in lines:
        items = line.split(', ')
        # find the frist possible item for the 2-item set
        for index1 in range(len(items) - 1):
            if items[index1] not in frequent_item1:
                continue
            # find the second possible item for the 2-item set
            for index2 in range(index1 + 1, len(items)):
                if items[index2] not in frequent_item1:
                    continue
                # arrange the items in alphabetic order
                itemset2 = (items[index1], items[index2])
                itemset2 = str(sorted(itemset2))
                supp_counts2[itemset2] += 1

    # generate L2
    frequent_item2 = set()
    for index in supp_counts2:
        if supp_counts2[index]/n_lines > min_sup:
            frequent_item2.add(index)

    print("Frequent 2-item set: \n", frequent_item2)

    #frequent 3-itemset L3
    supp_counts3 = defaultdict(int)

    # gernate C3 based on L2
    for line in lines:
        items = line.split(', ')
        for index1 in range(len(items)-2):
            if items[index1] not in frequent_item1:
                continue
            for index2 in range(index1+1, len(items)-1):
                if items[index2] not in frequent_item1:
                    continue
                # combine the first two items into a group for 2-itemset frequent pattern check
                index_group2_12 = (items[index1], items[index2])
                index_group2_12 = str(sorted(index_group2_12))
                # check if the 2-item set is frequent
                if index_group2_12 not in frequent_item2:
                    continue
                for index3 in range(index2+1, len(items)):
                    index_group2_13 = (items[index1], items[index3])
                    index_group2_13 = str(sorted(index_group2_13))
                    # check if the 2-itemset is frequent
                    if index_group2_13 not in frequent_item2:
                        continue
                    # combine the second and the third items into a group for 2-itemset frequent pattern check
                    index_group2_23 = (items[index2], items[index3])
                    index_group2_23 = str(sorted(index_group2_23))

                    # check if the 2-itemset if frequent
                    if index_group2_23 not in frequent_item2:
                        continue
                    itemset3 = (items[index1], items[index2], items[index3])
                    itemset3 = str(sorted(itemset3))
                    supp_counts3[itemset3] += 1

    # generate L3
    frequent_item3 = set()
    for index in supp_counts3:
        if supp_counts3[index]/n_lines > min_sup:
            frequent_item3.add(index)

    print("Frequent 3-item set: \n", frequent_item3)

#test the code
apriori("adult.data", min_sup=0.5)

Frequent 1-item set: 
 {'United-States', 'Private', 'Male', 'White', '<=50K\n', '0'}
Frequent 2-item set: 
 {"['<=50K\\n', 'Private']", "['<=50K\\n', 'White']", "['United-States', 'White']", "['0', 'Male']", "['0', 'United-States']", "['<=50K\\n', 'United-States']", "['0', 'Private']", "['0', 'White']", "['Male', 'United-States']", "['Private', 'White']", "['0', '0']", "['Male', 'White']", "['0', '<=50K\\n']", "['Private', 'United-States']"}
Frequent 3-item set: 
 {"['0', '<=50K\\n', 'United-States']", "['0', '0', 'United-States']", "['<=50K\\n', 'United-States', 'White']", "['0', 'Male', 'White']", "['0', '0', '<=50K\\n']", "['0', '<=50K\\n', 'White']", "['Male', 'United-States', 'White']", "['0', '0', 'Male']", "['0', 'Private', 'White']", "['0', 'Private', 'United-States']", "['Private', 'United-States', 'White']", "['0', 'Male', 'United-States']", "['0', '0', 'White']", "['0', '<=50K\\n', 'Private']", "['0', '0', 'Private']", "['0', 'United-States', 'White']"}


In the test, I set the minimum support as 0.5 and report the frequent itemset with 1, 2, and 3 items. The output shows that "United States", "Private", "Male", "White", "<=50K" and "0" (capital gain) is the requent 1-item set. In the 2-item set, there are many pairs that are meaningless in the sense to understand the relations between attributes. For instance, ("Male", "White") does not mean males are more likely to be white man. But some are potentially interesting, such as ("<=50K", "Private"), which means many people working in private sectors have income less than 50K. The 3-item set gives some more interesting results. For example, the ("Private", "United-States", "White") tuple can be explained as white American people have a large number of members working in private sectors, which can reflect some dimension of American employees.

### 2.2 Efficiency of Apriori with different parameters

In this subsection, I test the performance of the Apriori algorithm with different parameters and differnt sizes of dataset. I propose the following code chunk to track the running time of the Apriori algorithm without reporting specific results. The code is the same with the one in Section 2.1 exceptfor removing the "print" lines and adding a time tracker.

In [140]:
# construct the timer to evaluate apriori's efficiency
def ap_timer(data, min_sup):
    start_time = time.time()
    # read data
    with open(data) as df:
        lines = df.readlines()
    df.close()

    n_lines = sum(1 for line in open(data))  # generate the number of lines
    # print ("There are ", n_lines, "tuples in this data")
    # print (min_sup)

    # frequent 1-itemset L1
    # generate C1
    supp_counts1 = defaultdict(int)

    for line in lines:
        for item in line.split(', '):
            supp_counts1[item] += 1

    # generate L1
    frequent_item1 = set()
    for index in supp_counts1:
        if supp_counts1[index]/n_lines > min_sup:
            frequent_item1.add(index)

    # frequent 2-itemset L2
    supp_counts2 = defaultdict(int)

    # generate C2 based on L1
    for line in lines:
        items = line.split(', ')
        # find the frist possible item for the 2-itemset
        for index1 in range(len(items) - 1):
            if items[index1] not in frequent_item1:
                continue
            # find the second possible item for the 2-itemset
            for index2 in range(index1 + 1, len(items)):
                if items[index2] not in frequent_item1:
                    continue
                # arrange the items in alphabetic order
                itemset2 = (items[index1], items[index2])
                itemset2 = str(sorted(itemset2))
                supp_counts2[itemset2] += 1

    # generate L2
    frequent_item2 = set()
    for index in supp_counts2:
        if supp_counts2[index]/n_lines > min_sup:
            frequent_item2.add(index)


    #frequent 3-itemset L3
    supp_counts3 = defaultdict(int)

    # gernate C3 based on L2
    for line in lines:
        items = line.split(', ')
        for index1 in range(len(items)-2):
            if items[index1] not in frequent_item1:
                continue
            for index2 in range(index1+1, len(items)-1):
                if items[index2] not in frequent_item1:
                    continue
                # combine the first two items into a group for 2-itemset frequent pattern check
                index_group2_12 = (items[index1], items[index2])
                index_group2_12 = str(sorted(index_group2_12))
                # check if the 2-itemset is frequent
                if index_group2_12 not in frequent_item2:
                    continue
                for index3 in range(index2+1, len(items)):
                    index_group2_13 = (items[index1], items[index3])
                    index_group2_13 = str(sorted(index_group2_13))
                    # check if the 2-itemset is frequent
                    if index_group2_13 not in frequent_item2:
                        continue
                    # combine the second and the third items into a group for 2-itemset frequent pattern check
                    index_group2_23 = (items[index2], items[index3])
                    index_group2_23 = str(sorted(index_group2_23))

                    # check if the 2-itemset if frequent
                    if index_group2_23 not in frequent_item2:
                        continue
                    itemset3 = (items[index1], items[index2], items[index3])
                    itemset3 = str(sorted(itemset3))
                    supp_counts3[itemset3] += 1

    # generate L3
    frequent_item3 = set()
    for index in supp_counts3:
        if supp_counts3[index]/n_lines > min_sup:
            frequent_item3.add(index)

    print("--- %.6f seconds ---" % (time.time() - start_time))

In [141]:
ap_timer("adult.data", min_sup=0.3)
ap_timer("adult.data", min_sup=0.5)
ap_timer("adult.data", min_sup=0.7)

ap_timer("adult_1p.data", min_sup=0.5)
ap_timer("adult_10p.data", min_sup=0.5)
ap_timer("adult.data", min_sup=0.5)

--- 7.808020 seconds ---
--- 3.399386 seconds ---
--- 1.416005 seconds ---
--- 0.032830 seconds ---
--- 0.356331 seconds ---
--- 3.368015 seconds ---


First, I test the algorithm with the same data but different minimum support. The result shows that the running time is 7.8 seconds, 3.4 seconds and 1.4 seconds for minimum support 0.3, 0.5 and 0.7 respectively. The running time decreases as minimum support increases, because greater minumum support makes it throw away mroe candidates before scannding the whole dataset for the k-item set.

Alternatively, I test the performance of the algorithm with different data size. With 1%, 10% and the complete data, the running time is 0.03 seconds, 0.36 seconds and 3.37 seconds respectively. As the data size increases, the running time also increases.

### 2.3 Improved Apriori: using Hash-based technique

In this subssection, I propose the improved Apriori using hash-based technique. The hash-based technique distribute all possible candidate sets for k-item set into different buckets using hash formula while generating $L_{k-1}$. It utilizes the possible duplication in hash table to reduce the number of candidate set in the next scan. Note that although Han, Kamber, and Pei (2012) uses 7 as the hash parameter, there is no definite rule to pick an optimal parameter. In general, a too small parameter makes the algorithm useless because it cannot eliminate any of the candidate sets. In my practice, I tested several parameters and selected 59 when the minimum support is 0.3 in this example. The hash parameter helps to cut the number of candidates from 78 to 59. 

The code and the results are reported below:

In [7]:
def hash_ap(data, min_sup):
    start_time = time.time()

    with open(data) as df:
        lines = df.readlines()
    df.close()

    n_lines = sum(1 for line in open(data)) 

    supp_counts1 = defaultdict(int)
    H_index = defaultdict(int)
    hashTable = defaultdict(int32)

    for line in lines:
        for item in line.split(', '):
            supp_counts1[item] += 1

    # find frequent candidate for 1-itemset
    frequent_item1 = set()
    for index in supp_counts1:
        if supp_counts1[index]/n_lines > min_sup:
            frequent_item1.add(index)


    #Generate all 2-itemset based on L1
    L1 = list(frequent_item1)
    C2 = combinations(L1, 2)
    C2 = list(C2) #the length of C2 is 78

    #generate hash table and increase corresponding bucket counts
    for itemset in C2:
        item1 = itemset[0]
        index1 = L1.index(item1)
        item2 = itemset[1]
        index2 = L1.index(item2)
        #calculate hash index/address with hash function
        hindex = (index1*10 + index2) % 59
        H_index[itemset] = hindex #the dictionary store the itemset and corresponding index


        for line in lines:
            if itemset[0] in line and itemset[1] in line: 
                hashTable[hindex] += 1

    # generate a reduced 2-itemset for scan in the second pass
    candidate_hash2 = set()
    for key in hashTable:
        if hashTable[key]/n_lines > min_sup:
            candidate_hash2.add(key)
    #print(hashTable)


    H2 = set()
    for key in H_index:
        index = H_index.get(key)
        if index in candidate_hash2:
            H2.add(key)
        
    print("the length of C2 is", len(C2))
    print("the length of H2 is", len(H2))

    supp_counts2= defaultdict(int) #the dictionary to track the counts of 2-itemset

    #find the frequent 2-itemset candidates
    for line in lines:
        items = line.split(', ')
        #find the frist possible item for the 2-itemset
        for index1 in range(len(items) -1): 
            if items[index1] not in frequent_item1:
                continue
            #find the second possible item for the 2-itemset
            for index2 in range(index1+ 1, len(items)):
                if items[index2] not in frequent_item1:
                    continue
                #arrange the items in alphabetic order
                itemset2 = (items[index1], items[index2])
                itemset2 = str(sorted(itemset2))
                supp_counts2[itemset2] += 1

    #find the frequent 2-itemset
    frequent_item2 = set()
    for index in supp_counts2:
        if supp_counts2[index]/n_lines > min_sup:
            frequent_item2.add(index)

    print(frequent_item2)
    print("--- %.6f seconds ---" % (time.time() - start_time))


hash_ap("adult.data", min_sup=0.3)

the length of C2 is 78
the length of H2 is 59
{"['0', 'Never-married']", "['<=50K\\n', 'White']", "['Male', 'Private']", "['Husband', 'United-States']", "['40', 'Male']", "['0', '<=50K\\n']", "['Married-civ-spouse', 'White']", "['0', 'Married-civ-spouse']", "['0', 'Male']", "['<=50K\\n', 'Never-married']", "['<=50K\\n', 'Male']", "['40', 'White']", "['Male', 'United-States']", "['Married-civ-spouse', 'United-States']", "['0', 'Female']", "['Husband', 'White']", "['0', '9']", "['9', 'HS-grad']", "['0', 'Private']", "['<=50K\\n', 'United-States']", "['40', '<=50K\\n']", "['0', '40']", "['0', '0']", "['Private', 'United-States']", "['40', 'Private']", "['<=50K\\n', 'Private']", "['Husband', 'Male']", "['0', 'United-States']", "['United-States', 'White']", "['40', 'United-States']", "['Private', 'White']", "['Husband', 'Married-civ-spouse']", "['0', 'Husband']", "['0', 'White']", "['0', 'HS-grad']", "['Male', 'Married-civ-spouse']", "['Male', 'White']"}
--- 3.438760 seconds ---


To compare the efficiency of the two Apriori algorithms, similar to section 2.2, I propose the following timer for hash-based Apriori. Then I run several tests with differnt parameters to compare their performance.

In [143]:
def hash_timer(data, min_sup):
    start_time = time.time()

    with open(data) as df:
        lines = df.readlines()
    df.close()

    n_lines = sum(1 for line in open(data)) 

    supp_counts1 = defaultdict(int)
    H_index = defaultdict(int)
    hashTable = defaultdict(int32)

    for line in lines:
        for item in line.split(', '):
            supp_counts1[item] += 1

    # find frequent candidate for 1-itemset
    frequent_item1 = set()
    for index in supp_counts1:
        if supp_counts1[index]/n_lines > min_sup:
            frequent_item1.add(index)


    #Generate all 2-itemset based on L1
    L1 = list(frequent_item1)
    C2 = combinations(L1, 2)
    C2 = list(C2) #the length of C2 is 78

    #generate hash table and increase corresponding bucket counts
    for itemset in C2:
        item1 = itemset[0]
        index1 = L1.index(item1)
        item2 = itemset[1]
        index2 = L1.index(item2)
        #calculate hash index/address with hash function
        hindex = (index1*10 + index2) % 59
        H_index[itemset] = hindex #the dictionary store the itemset and corresponding index


        for line in lines:
            if itemset[0] in line and itemset[1] in line: 
                hashTable[hindex] += 1

    # generate a reduced 2-itemset for scan in the second pass
    candidate_hash2 = set()
    for key in hashTable:
        if hashTable[key]/n_lines > min_sup:
            candidate_hash2.add(key)
    #print(hashTable)


    H2 = set()
    for key in H_index:
        index = H_index.get(key)
        if index in candidate_hash2:
            H2.add(key)
        
    #print("the length of C2 is", len(C2))
    #print("the length of H2 is", len(H2))

    supp_counts2= defaultdict(int) #the dictionary to track the counts of 2-itemset

    #find the frequent 2-itemset candidates
    for line in lines:
        items = line.split(', ')
        #find the frist possible item for the 2-itemset
        for index1 in range(len(items) -1): 
            if items[index1] not in frequent_item1:
                continue
            #find the second possible item for the 2-itemset
            for index2 in range(index1+ 1, len(items)):
                if items[index2] not in frequent_item1:
                    continue
                #arrange the items in alphabetic order
                itemset2 = (items[index1], items[index2])
                itemset2 = str(sorted(itemset2))
                supp_counts2[itemset2] += 1

    #find the frequent 2-itemset
    frequent_item2 = set()
    for index in supp_counts2:
        if supp_counts2[index]/n_lines > min_sup:
            frequent_item2.add(index)

    #print(frequent_item2)
    print("--- %.6f seconds ---" % (time.time() - start_time))

In [144]:
hash_timer("adult.data", min_sup=0.3)
hash_timer("adult.data", min_sup=0.5)
hash_timer("adult.data", min_sup=0.7)

--- 2.402912 seconds ---
--- 0.879161 seconds ---
--- 0.615119 seconds ---


Compare the running time of Apriori in section 2.2 and the running time of hash-based Apriori in the section, It is shown that the efficiency of hash-based Apriori is better than the original one.

![image](Figure1.png)

## 3 FP-Growth

In this section, I describe the FP-Growth algorithm, code and the performance evaluation. FP-Growth borrows the data structure of trees to avoid generateting a large pool of candidates. The algorithm of FP-Growth can be described as follows:

1. Find frequent items
   - Find all frequent 1-item set. 
   - Sort all the frequent 1-item set in frequency in descending order, at the same time, delete the items whose frequency is smaller than min_sup
   - For each tuple, reorder the items by the order in the sorted frequent 1-item set

2. Construct FP tree
   - Create a node object to track the position of items
   - Create a tree object with root = “null”. 
   Fill the tree with frequent items and their support counts, and assign each node a key
   For each line in the data, sort the items in the order of sorted frequent items
   Create a node table to track the order of node

3. Generate conditional FP tree
   - Given the FP tree and the stored nodes, construct a conditional FP tree
   - Construct a set containing all the suffix by the order of nodes

4. Generate all frequent patterns based on conditional FP tree

The code to apply the FP-Growth algorithm is presented in 3.1. To test the algorithm, I use the 10% subset of UCI Adult Census Dataset. I did not use the complete dataset becasue FP-Growth finds all frequent itemsets until hit the maximum length of all attributes. Thus it can take a long time to test the function. Of the purpose to test the function and performance efficiently, I use the 10% subset of the data.


### 3.1 Main method

In [11]:
import pandas as pd
from collections import defaultdict
import itertools

def readdata(data):
    with open(data) as df:
        newdata = []
        lines = df.readlines()
        for line in lines:
            newdata.append(line.split(", "))
    df.close()
    return newdata

# test data
min_sup = 0.5

test_data2 = readdata("adult_10p.data")
obs = sum(1 for line in open("adult_10p.data"))


# Begin FP-Groth Algorithm
# construct the node class, which stores the item, support counts, children node and parent node

class Node:
    def __init__(self, item, item_count=0, parent=None):
        self.item = item
        self.item_count = item_count
        self.parent = parent
        self.children = {}

# construct the FP tree


class FPtree:
    def __init__(self, data, min_sup, obs):
        self.data = data
        self.min_sup = min_sup
        self.root = Node(item='null')
        self.obs = obs


        nodeTable = []

        # find all the frequent 1-itemset in the data
        sup_counts = defaultdict(int)
        freqitem = set()
        for line in data:
            for items in line:
                sup_counts[items] += 1

        for item in sup_counts:
            if sup_counts[item]/obs >= self.min_sup:
                freqitem.add(item)
        # sort the frequent items
        freqSort = sorted(sup_counts.items(), key=lambda x: (-x[1], x[0]))
        for i in freqSort:
            if i[0] not in freqitem:
                freqSort.remove(i)

        # construct the FP tree
        # sort the items in each line
        for line in data:
            lineSort = []
            for i in freqSort:
                if i[0] in line:
                    lineSort.append(i[0])
            line = lineSort
            # print(lineSort)
            # insert the items into the FP tree
            R = self.root
            for item in line:
                if item not in R.children.keys():
                    R.children[item] = Node(
                        item=item, item_count=1, parent=R)
                else:
                    R.children[item].item_count += 1
                    R.children[item].parent = R

                R = R.children[item]
                nodeTable.append(R)

        # construct conditional tree
        # find the largest key and starting from the node with this key

        def condline(N):
            condfpline = []
            if N.parent == None:
                return None
            while N.parent.item != 'null':
                P = N.parent
                condfpline.append(P.item)
                N = N.parent
            return condfpline

        condset = []
        for node in nodeTable:
            n = condline(node)
            condset.append(n)

        # find all frequent pattern
        freq_patt = []
        for item in freqSort:
            condtreeset = []
            i = 0
            for node in nodeTable:
                if node.item == item[0]:
                    condtreeset.append(condset[i])
                i += 1
            # print(condtreeset)

            cdcount = defaultdict(int)
            condFP = []
            for line in condtreeset:
                for cand in line:
                    cdcount[cand] += 1
            for cand in cdcount:
                if cdcount[cand]/obs >= min_sup:
                    condFP.append(cand)
            if len(condFP):
                condFP.append(item[0])
               # print(condFP)
                c = len(condFP)
                while c > 3:
                    pattern = list(itertools.combinations(condFP, c))
                    for t in pattern:
                        if item[0] not in str(t):
                            pattern.remove(t)
                    print(pattern)
                    # freq_patt.append(pattern)
                    c = c - 1


FPtree(test_data2, min_sup, obs)

[]
[('<=50K\n', 'White', 'United-States', '0', 'Private')]
[('<=50K\n', 'White', 'United-States', 'Private'), ('<=50K\n', 'White', '0', 'Private'), ('<=50K\n', 'United-States', '0', 'Private'), ('White', 'United-States', '0', 'Private')]
[('White', 'United-States', '0', 'Male')]


I set the minimum support to be 0.5 to test the FP-algorithm. It returns all the frequent itemset in the output. Some of the results reveal possible correlations among attributes. For example, ('<=50K\n', 'United-States', '0', 'Private') suggests there are many Americans working in private sectors having income less than 50K and 0 capital gain.

### 3.2 Efficiency of FP-Growth with different parameters
To evaluate the efficiency of FP-Growth algorithm, similar to section 2.2 and 2.3, I construct a timer to track the running time of FP-growth with different parameters. To save the space, I put the code and the output in the Appendix. The results show that the running time is 13.44 seconds, 11.45 seconds and 10.14 seconds with the minimum support equals to 0.1, 0.3, and 0.6 respectively.

## 4 Compare Apriori and FP-Growth

In this section, I compare the performance of the Apriori, Hash-based Apriori and the FP-Growth. I did not use the UCI Adult Census Database, however, because while Apriori finds frequent itemset until 3-item set, the FP-Growth finds all frequent itemset. Because there are many attributes and tuples in the UCI Adult Census Database, the output are compeltely different for the two algorithms. To avoid comparing apples with oranges, I use the data used in Han, Kamber, and Pei (2012). The output of this data is the same for all the algorothms. Although the dataset is quite small, it is still enough to reveal the efficiency of different algotithms. The specific dataset is shown in Appendix.

In [146]:
start_time = time.time()
FPtree("test.data", min_sup=2/9, obs=9)
print("--- %.6f seconds ---" % (time.time() - start_time))

start_time = time.time()
FPtree("test.data", min_sup=5/9, obs=9)
print("--- %.6f seconds ---" % (time.time() - start_time))

start_time = time.time()
FPtree("test.data", min_sup=7/9, obs=9)
print("--- %.6f seconds ---" % (time.time() - start_time))

--- 0.000267 seconds ---
--- 0.000219 seconds ---
--- 0.000179 seconds ---


In [147]:
ap_timer("test.data", min_sup=2/9)
ap_timer("test.data", min_sup=5/9)
ap_timer("test.data", min_sup=7/9)

--- 0.002349 seconds ---
--- 0.001937 seconds ---
--- 0.000260 seconds ---


In [148]:
hash_timer("test.data", min_sup=2/9)
hash_timer("test.data", min_sup=5/9)
hash_timer("test.data", min_sup=7/9)

--- 0.001444 seconds ---
--- 0.000842 seconds ---
--- 0.001042 seconds ---


I combine the results of the three timers to show the efficiency of the three algorithms using the timers. Figure 2 shows the effciency comparison of the algorithms.

![image](Figure2.png)

According to the figure, FP-Growth is much faster than Apriori and Hash-based Apriori. This is due to the lower computational complexity of FP-Growth. 

## Conclusion

In this programming project, I practice the application of Apriori, Hash-based Apriori and the FP-Growth algorithm using the UCI Adult Census Database. The output reports some results that may reflect correlations among American people's demographic features. The report also presents the performance of the three algorithms. In general, Hash-based Apriori reduces the running time significantly and FP-Growth is much more efficient than Apriori. This implies FP-Growth should be optimal in general cases rather than Apriori.

## References
Han, Kamber, M., & Pei, J. (2012). Data mining concepts and techniques / Jiawei Han, Micheline Kamber, Jian Pei. (3rd ed.). Elsevier.

UCI Machine Learning Repository: Adult Data Set. (n.d.). Retrieved October 6, 2022, from http://archive.ics.uci.edu/ml/datasets/Adult

## Appendix

In [16]:
import pandas as pd
from collections import defaultdict
import itertools

def readdata(data):
    with open(data) as df:
        newdata = []
        lines = df.readlines()
        for line in lines:
            newdata.append(line.split(", "))
    df.close()
    return newdata

# test data


test_data = [['I1', 'I2', 'I5'],
             ['I2', 'I4'],
             ['I2', 'I3'],
             ['I1', 'I2', 'I4'],
             ['I1', 'I3'],
             ['I2', 'I3'],
             ['I1', 'I3'],
             ['I1', 'I2', 'I3', 'I5'],
             ['I1', 'I2', 'I3', 'I6']]

test_data2 = readdata("adult_10p.data")
obs = sum(1 for line in open("adult_10p.data"))


# Begin FP-Groth Algorithm
# construct the node class, which stores the item, support counts, children node and parent node

class Node:
    def __init__(self, item, item_count=0, parent=None):
        self.item = item
        self.item_count = item_count
        self.parent = parent
        self.children = {}

# construct the FP tree


class FPtimer:
    def __init__(self, data, min_sup, obs):
        self.data = data
        self.min_sup = min_sup
        self.root = Node(item='null')
        self.obs = obs


        nodeTable = []

        # find all the frequent 1-itemset in the data
        sup_counts = defaultdict(int)
        freqitem = set()
        for line in data:
            for items in line:
                sup_counts[items] += 1

        for item in sup_counts:
            if sup_counts[item]/obs >= self.min_sup:
                freqitem.add(item)
        # sort the frequent items
        freqSort = sorted(sup_counts.items(), key=lambda x: (-x[1], x[0]))
        for i in freqSort:
            if i[0] not in freqitem:
                freqSort.remove(i)

        # construct the FP tree
        # sort the items in each line
        for line in data:
            lineSort = []
            for i in freqSort:
                if i[0] in line:
                    lineSort.append(i[0])
            line = lineSort
            # print(lineSort)
            # insert the items into the FP tree
            R = self.root
            for item in line:
                if item not in R.children.keys():
                    R.children[item] = Node(
                        item=item, item_count=1, parent=R)
                else:
                    R.children[item].item_count += 1
                    R.children[item].parent = R

                R = R.children[item]
                nodeTable.append(R)

        # construct conditional tree
        # find the largest key and starting from the node with this key

        def condline(N):
            condfpline = []
            if N.parent == None:
                return None
            while N.parent.item != 'null':
                P = N.parent
                condfpline.append(P.item)
                N = N.parent
            return condfpline

        condset = []
        for node in nodeTable:
            n = condline(node)
            condset.append(n)

        # find all frequent pattern
        freq_patt = []
        for item in freqSort:
            condtreeset = []
            i = 0
            for node in nodeTable:
                if node.item == item[0]:
                    condtreeset.append(condset[i])
                i += 1
            # print(condtreeset)

            cdcount = defaultdict(int)
            condFP = []
            for line in condtreeset:
                for cand in line:
                    cdcount[cand] += 1
            for cand in cdcount:
                if cdcount[cand]/obs >= min_sup:
                    condFP.append(cand)
            if len(condFP):
                condFP.append(item[0])
               # print(condFP)
                c = len(condFP)
                while c > 3:
                    pattern = list(itertools.combinations(condFP, c))
                    for t in pattern:
                        if item[0] not in str(t):
                            pattern.remove(t)
                    #print(pattern)
                    # freq_patt.append(pattern)
                    c = c - 1


start_time = time.time()
min_sup = 0.1
FPtimer(test_data2, min_sup, obs=obs)
print("--- %.6f seconds ---" % (time.time() - start_time))

start_time = time.time()
min_sup = 0.3
FPtimer(test_data2, min_sup, obs=obs)
print("--- %.6f seconds ---" % (time.time() - start_time))

start_time = time.time()
min_sup = 0.6
FPtimer(test_data2, min_sup, obs=obs)
print("--- %.6f seconds ---" % (time.time() - start_time))

--- 13.438151 seconds ---
--- 11.453575 seconds ---
--- 10.137549 seconds ---


In [17]:
min_sup = 2/9

test_data = [['I1', 'I2', 'I5'],
             ['I2', 'I4'],
             ['I2', 'I3'],
             ['I1', 'I2', 'I4'],
             ['I1', 'I3'],
             ['I2', 'I3'],
             ['I1', 'I3'],
             ['I1', 'I2', 'I3', 'I5'],
             ['I1', 'I2', 'I3', 'I6']]